In [1]:
import importlib
import ifimage_tools
importlib.reload(ifimage_tools)
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from preprocessing import SamplePreprocessor,batch_process_zip_rois,batch_rename,mask_to_rois_zip

auto.py (21): IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
import os
import numpy as np
from stardist.matching import matching
image_dir = "Reorgnized Ground Truth"
masks_dir = "Reorgnized Ground Truth/mask"
dataset = ifimage_tools.IfImageDataset(image_dir, masks_dir, {})
dataset.load_data()

old_version_sample_ids = ["6390", "8408", "8406", "8405v2", "8405", "8407"]
for sample_id in old_version_sample_ids:
    if sample_id in dataset.samples:
        del dataset.samples[sample_id]

In [ ]:
# to_process = []
# for sample_id, sample in dataset.samples.items():
#     has_cellbodies = (sample.cellbodies_mask is not None) or (sample.cellbodies_multimask is not None)
#     has_dapi_multi = sample.dapi_multi_mask is not None
#     if has_cellbodies and not has_dapi_multi:
#         to_process.append(sample_id)

# print(f"—— 一共发现 {len(to_process)} 个样本，需要先跑 apply_nuc_pipeline，然后生成 ROI：{to_process}")


—— 一共发现 60 个样本，需要先跑 apply_nuc_pipeline，然后生成 ROI：['8550', '8917', '5191', '6833', '10166', '9106', '4642', '5792', '10061', '1110', '7925', '5863', '1112', '7739', '5059', '7113', '3569', '7685', '8224', '4515', '7870', '6790', '7962', '1111', '8746', '3532', '3999', '3527', '7071', '5923', '5789', '5794', '9755', '6020', '9170', '10015', '1116', '4238', '1114', '5795', '4548', '8517', '9783', '1108', '8310', '9472', '4736', '7319', '9866', '6212', '8942', '1113', '6748', '1115', '5410', '6523', '6466', '4683', '4319', '1109']


In [ ]:
# 输出目录
output_zip_dir = "roi_zips"
os.makedirs(output_zip_dir, exist_ok=True)

for sid in to_process:
    sample = dataset.samples[sid]
    print(f"\n[INFO] 开始处理样本：{sid}")

    # 1. 调用 apply_nuc_pipeline 生成 dapi_multi_mask
    print(f"  • 运行 apply_nuc_pipeline...")
    sample.apply_nuc_pipeline()

    if sample.masks is None:
        print(f"  [WARN] 样本 {sid} 在运行 apply_nuc_pipeline 后仍然没有 dapi_multi_mask，跳过。")
        continue

    dapi_mask = sample.masks.copy()["cellposeSAM"]
    # 如果是 0/1 二值图，就先做 connected-components 标号
    from skimage.measure import label as sklabel
    if dapi_mask.max() <= 1:
        dapi_mask = sklabel(dapi_mask)

    # 3. 调用 mask_to_rois_zip，把 dapi_mask 转成 ImageJ ROI 并打包
    celltype=sample.celltype
    zip_path = os.path.join(output_zip_dir, f"{celltype}_{sid}_nuclei_roi.zip")
    print(f"  • 生成 ROI ZIP: {zip_path} ...")
    mask_to_rois_zip(dapi_mask, zip_path)

    print(f"  [OK] 样本 {sid} 的 nuclei ROI ZIP 已保存。")

print("\n全部样本处理完毕，请到 “roi_zips” 目录查看各个 *_nuclei_roi.zip 文件。")


[INFO] 开始处理样本：8550
  • 运行 apply_nuc_pipeline...


channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


Applied cellposeSAM
  • 生成 ROI ZIP: roi_zips/8550_nuclei_roi.zip ...
ROI for label 1 saved as 'roi_1.roi'.
ROI for label 2 saved as 'roi_2.roi'.
ROI for label 3 saved as 'roi_3.roi'.
ROI for label 4 saved as 'roi_4.roi'.
ROI for label 5 saved as 'roi_5.roi'.
ROI for label 6 saved as 'roi_6.roi'.
ROI for label 7 saved as 'roi_7.roi'.
ROI for label 8 saved as 'roi_8.roi'.
ROI for label 9 saved as 'roi_9.roi'.
ROI for label 10 saved as 'roi_10.roi'.
ROI for label 11 saved as 'roi_11.roi'.
ROI for label 12 saved as 'roi_12.roi'.
ROI for label 13 saved as 'roi_13.roi'.
ROI for label 14 saved as 'roi_14.roi'.
ROI for label 15 saved as 'roi_15.roi'.
ROI for label 16 saved as 'roi_16.roi'.
ROI for label 17 saved as 'roi_17.roi'.
ROI for label 18 saved as 'roi_18.roi'.
ROI for label 19 saved as 'roi_19.roi'.
ROI for label 20 saved as 'roi_20.roi'.
ROI for label 21 saved as 'roi_21.roi'.
ROI for label 22 saved as 'roi_22.roi'.
ROI for label 23 saved as 'roi_23.roi'.
ROI for label 24 saved as 'ro

channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used


KeyboardInterrupt: 